# **Estimating the value of mathematical constant pi (π)**

# Import dependencies

In [ ]:
import math, random, numpy as np

# Introduction to Monte Carlo simulations

## Part 1 -- Simulating a six sided regular fair dice

### Hyper-dimensional dice

In [ ]:
def random_dice_throw(
        num_dice_faces:float=2.0,
        return_dice_face:bool=False
      )->'Either just a boolean value or a tuple with boolean and float values':
    '''
       A function that returns the win probability for a given                                    \
         hyper-dimensional dice of "num_dice_faces".

           Arguments of the function:
               num_dice_faces:float  -> Accepts fractional values for "num_dice_faces".           \
                                          Defaults to 2 if nothing is specified,                  \
                                          mimicking a coin-toss win probability.                  \
               return_dice_face:bool -> Boolean flag that determines whether or not               \
                                          a dice face is returned by the function.                \
                                          An integer output can be computed using                 \
                                          the dice face probability through the ceiling function.

           The function returns either just a boolean value when return_probabilities = False,    \
             or returns a tuple with boolean and float values when return_probabilities = True.
    '''
    assert num_dice_faces > 1, 'Specify a value > 1 for "num_faces". Defaults to 2 if nothing is specified, mimicking a coin-toss win probability ...'
    rnd_prob = random.SystemRandom().random()
    bool_prob = bool(math.ceil(1 / rnd_prob) <= num_dice_faces)
    if return_dice_face:
        out_prob = bool_prob, rnd_prob * num_dice_faces; del rnd_prob, bool_prob
        return out_prob
    del rnd_prob
    return bool_prob

In [ ]:
true_prob = 0
num_throws = 100000
for i in range(num_throws):
    bool_prob, dice_face = random_dice_throw(num_dice_faces=6, return_dice_face=True)
    if bool_prob:
        true_prob += 1
    if (i + 1) % (num_throws * 0.1) == 0:
        print(f'Boolean outcome for the {i + 1} attempt: {bool_prob} ...')
        print(f'Dice face outcome for the {i + 1} attempt: {math.ceil(dice_face)} ...')
        print(f'Expected cumulative probability for the dice throw outcome to be ```True```: {true_prob / (i + 1)} ...')
        print(f'Cumulative probability for the dice throw outcome to be ```True```: {true_prob / (i + 1)} ...')
        print('\n')
print(true_prob / num_throws)

## Part 2 -- Modeling roulette wheel using Monte Carlo simulations

### Fair Roulette
A function that mimicks the behavior of a roulette wheel with 36 slots, each uniquely numbered between 1 through 36.

In [ ]:
class FairRoulette():
    def __init__(self):
        self.pockets     = [i for i in range(1, 37)]
        self.prob        = None
        self.ball        = None
        self.win         = False 
        self.pocket_odds = len(self.pockets) - 1
    def spin(self):
        self.bool_prob, self.prob = random_dice_throw(num_dice_faces=len(self.pockets), return_dice_face=True)
        self.ball = math.ceil(self.prob)
    def bet_pocket(self, bet_pocket, amount):
        assert isinstance(amount, int) or isinstance(amount, float), f'Invalid input for the betting amount: {amount} ...'
        assert isinstance(bet_pocket, int), f'Expected an integer value as a bet for the pocket number, instead received: {bet_pocket} ...'
        assert 0 < bet_pocket < 37, f'Invalid bet placed for the pocket number: {bet_pocket} ...'
        if bet_pocket == self.ball:
            self.win = True
            return amount * self.pocket_odds, self.ball, self.win
        return -amount, self.ball, self.win
    def __str__(self):
        return 'Fair Roulette'

In [ ]:
fair_roulette = FairRoulette()
fair_roulette.spin()

num_games = 1000000
bet_amount_per_game = 100
bet_pocket = 1

print(fair_roulette.bet_pocket(bet_pocket, bet_amount_per_game))

final_outcome_amount = 0
total_money_spent = 0
for i in range(num_games):
    fair_roulette.spin()
    total_money_spent += bet_amount_per_game
    outcome = fair_roulette.bet_pocket(bet_pocket, bet_amount_per_game)
    final_outcome_amount += outcome[0]

total_money_lost = total_money_spent - final_outcome_amount
print(final_outcome_amount, total_money_spent, total_money_lost)

# Estimating π using algebraic techniques

## Part 1 -- Using [Wallis integrals](https://en.wikipedia.org/wiki/Wallis%27_integrals) for estimating π

In [ ]:
def wallis_pi_estimator(pi_init=None, pi_init_step=None, num_steps=1000000):
    pi = np.array(2).astype('float64')

    if pi_init is not None and pi_init_step is not None:
        pi =  np.array(2).astype('float64') * np.array(pi_init).astype('float64')
    else:
        pi_init_step = 1

    for i in range(pi_init_step, num_steps + pi_init_step):
        i = np.array(i).astype('float64')
        pi *= ((np.array(2).astype('float64') * i) * (np.array(2).astype('float64') * i)) / (((np.array(2).astype('float64') * i) - 1) * ((np.array(2).astype('float64') * i) + 1))

    return pi

In [ ]:
pi_w = wallis_pi_estimator(
           pi_init=None,
           pi_init_step=None,
           num_steps=1000000
         )

In [ ]:
print(pi_w, math.pi, pi_w - math.pi)

## Part 2 -- [Ramanujan technique](https://en.wikipedia.org/wiki/Ramanujan%E2%80%93Sato_series) for estimating π

In [ ]:
def factorial(n):
    factorial_func = np.vectorize(math.factorial)
    return factorial_func(np.array(n).astype('int64'))

In [ ]:
def ramanujan_sato_pi_estimator(num_steps=1):
    alpha = (2 * np.power(np.array(2).astype('float64'), np.array(0.5).astype('float64'))) / np.power(np.array(99).astype('int64'), np.array(2).astype('int64'))

    rs_sum = 0
    for k in range(num_steps):
        ff = factorial( np.array(4 * k).astype('int64') ) / np.power(np.array(factorial(k)).astype('int64'), 4)

        rs_denom = np.power(np.array(396).astype('int64'), np.array(4 * k).astype('int64'))
        rs_numer = (np.array(26390).astype('int64') * np.array(k).astype('int64')) + np.array(1103).astype('int64')

        rs_sum  += ff * (rs_numer / rs_denom)

    pi = 1 / (alpha * rs_sum)

    return pi

In [ ]:
pi_r = ramanujan_sato_pi_estimator(2)

In [ ]:
print(pi_r, math.pi, pi_r - math.pi)

## Part 3 -- Using [Basel problem](https://en.wikipedia.org/wiki/Basel_problem)

In [ ]:
def basel_pi_estimator(num_steps=100):
    s = 1
    for k in range(2, num_steps + 2):
        s += 1 / (k * k) 

    pi = np.power(s * 6, 0.5)

    return pi

In [ ]:
pi_b = basel_pi_estimator(51200)

In [ ]:
print(pi_b, math.pi, pi_b - math.pi)

# Using Monte Carlo simulations

## Part 1 -- Using random points inside a square

In [ ]:
def monte_carlo_pi_sampler(
        num_mc_sims:int=2500,
        num_pi_sims:int=7500,
        max_int:int=262144
    )->tuple:
    '''
       A sampling function for estimating the value of π using Monte-Carlo simulations.

           Arguments of the function:
               num_mc_sims:int -> Number of Monte-Carlo simulations.
               num_pi_sims:int -> Number of π estimation simulations with a given radius.
               max_int:int     -> Maximum value for the radius to be used in π estimations simulations.

           Returns a tuple of three integers:                                                          \
             (Number of points inside the circle,                                                      \
              Number of points inside the largest square within the circle,                            \
              Number of points inside the smallest square outside the circle)
    '''
    assert isinstance(num_mc_sims, int), f'Expected an integer input for num_pi_sims, instead received: {num_pi_sims} ...'
    assert isinstance(num_pi_sims, int), f'Expected an integer input for num_pi_sims, instead received: {num_pi_sims} ...'
    assert isinstance(max_int, int),     f'Expected an integer input for max_int, instead received: {num_pi_sims} ...'
    in_points, sq_in_points, total_points = 0, 0, 0

    for s in range(num_pi_sims):
        rand_num = random.SystemRandom().uniform(1, max_int)
    
        sq_len = random.SystemRandom().uniform(0, max_int)
        sq_len = random.SystemRandom().choice([-sq_len, sq_len])

        x_min = random.SystemRandom().uniform(-rand_num, rand_num)
        x_max = x_min + sq_len

        sq_len = random.SystemRandom().choice([-sq_len, sq_len])
    
        y_min = random.SystemRandom().uniform(-rand_num, rand_num)
        y_max = y_min + sq_len
    
        x_list = np.asarray([[random.SystemRandom().uniform(x_min, x_max), x_min] for i in range(num_mc_sims)])
        y_list = np.asarray([[random.SystemRandom().uniform(y_min, y_max), y_min] for j in range(num_mc_sims)])

        c_dist        = (np.power(np.asarray(x_list)[:,0] - np.asarray(x_list)[:,1], 2) + 
                         np.power(np.asarray(y_list)[:,0] - np.asarray(y_list)[:,1], 2)) / math.pow(sq_len, 2)
        in_points    += np.sum(np.where(c_dist <= 1.0 , 1.0, 0.0), axis=0)
        total_points += len(c_dist)

        concat_list   = np.transpose(
                            np.array(
                                [np.abs(x_list[:,0] - x_list[:,1]), 
                                 np.abs(y_list[:,0] - y_list[:,1])]
                              )
                          )

        sq_in_points += np.sum(
                            np.where(
                                concat_list[:,0] <= np.abs(sq_len) / math.pow(2, 0.5), 1, 0
                              ) * \
                            np.where(
                                concat_list[:,1] <= np.abs(sq_len) / math.pow(2, 0.5), 1, 0
                              )
                          )

        if (s + 1) % (num_pi_sims * 0.1) == 0.0:
            print(f'Completed Monte Carlo pi simulation step: {s + 1} out of {num_pi_sims} ...')

    return in_points, sq_in_points, total_points

In [ ]:
def monte_carlo_pi_estimator(
        in_points:int,
        sq_in_points:int,
        total_points:int
      )->float:
    '''
       A function to approximate the value of π using Monte-Carlo simulation samples.

           Arguments of the function:
               in_points:int    -> Number of points inside the circle.
               sq_in_points:int -> Number of points inside the largest square within the circle. 
               total_points:int -> Number of points inside the smallest square outside the circle.

           Returns a float corresponding to the approximate value of π.
    '''
    pi_ratio    = (in_points / total_points) * 4
    pi_sq       = (in_points / sq_in_points) * 2
    pi_rem_area = (2 + (4 * ((in_points - sq_in_points) / total_points)))

    pi_out = (pi_ratio + pi_sq + pi_rem_area) / 3

    return pi_out

## Riemann-Remanan error correction
* An error correction is applied to the Monte-Carlo simulation output using Riemann-Remanan stochastic integration.
* The error correction works by performing a stochastic integration over a sentinel population, which returns an error estimate of the random sampling process. 
* An output range is computed using the error estimation to compensate for the sampling error.
* The final output is generated using a final stochastic sampling step.

In [ ]:
def riemann_remanan_pi_estimator(
        in_points:int,
        sq_in_points:int,
        total_points:int,
        num_steps:int=50000,
        pi_lower_bound:float=basel_pi_estimator(10000),
        pi_upper_bound:float=ramanujan_sato_pi_estimator(1),
        num_reties:int=5000
      )->float:
    '''
       Error correction using Riemann-Remanan stochastic integration.

         * An error correction is applied to the Monte-Carlo simulation output using Riemann-Remanan stochastic integration.
         * The error correction works by performing a stochastic integration over a sentinel population,                          \
             which returns an error estimate of the random sampling process.
         * An output range is computed using the error estimation to compensate for the sampling error.
         * The final output is generated using a final stochastic sampling step.

           Arguments of the function:
               in_points:int        -> Number of points inside the area of a π / 4 radians segment of the circle of radius r.
               sq_in_points:int     -> Number of points inside the square area of diagonal length r,                              \
                                         within the segemnt of the circle.
               total_points:int     -> Total number of points inside the square area of length r.
               num_steps:int        -> Number of Monte-Carlo simulations sampling steps.
               pi_lower_bound:float -> Lower bound value of the sampling filter.
               pi_upper_bound:float -> Upper bound value of the sampling filter.
               num_reties:int       -> Total retries when the filter output is None.

           Returns a float value corresponding to the approximation of pi. 
    '''
    sq_area_ratio = 1 / 2
    riemann_remanan_correction_list = [(sq_in_points / total_points) * 2]

    in_points_rr_list, sq_in_points_rr_list, total_points_rr_list = [], [], []
    for riemann_remanan_correction in riemann_remanan_correction_list:
        in_points_rr_list    += [in_points    * riemann_remanan_correction, in_points / riemann_remanan_correction]
        sq_in_points_rr_list += [sq_in_points * riemann_remanan_correction, sq_in_points / riemann_remanan_correction]
        total_points_rr_list += [total_points * riemann_remanan_correction, total_points / riemann_remanan_correction]

    in_points_rr_min, in_points_rr_max       = np.min(in_points_rr_list),    np.max(in_points_rr_list)
    sq_in_points_rr_min, sq_in_points_rr_max = np.min(sq_in_points_rr_list), np.max(sq_in_points_rr_list)
    total_points_rr_min, total_points_rr_max = np.min(total_points_rr_list), np.max(total_points_rr_list)

    pi, total_steps, num_retry_steps = 0, 0, 0
    while total_steps == 0:
        num_retry_steps += 1
        for i in range(num_steps):
            in_points_rr_mean = np.mean(in_points_rr_list)
            sq_in_points_rr_mean = np.mean(sq_in_points_rr_list)
            total_points_rr_mean = np.mean(total_points_rr_list)
            pi_rand = monte_carlo_pi_estimator(
                          (random.SystemRandom().choice(in_points_rr_list) + in_points_rr_mean) / 2, 
                          (random.SystemRandom().choice(sq_in_points_rr_list) + sq_in_points_rr_mean) / 2,  
                          (random.SystemRandom().choice(total_points_rr_list) + total_points_rr_mean) / 2
                        )
            if pi_upper_bound is None or pi_lower_bound is None:
                pi += pi_rand
                total_steps += 1
            elif pi_rand > pi_lower_bound and pi_rand < pi_upper_bound:
                pi += pi_rand
                total_steps += 1

        if num_retry_steps > num_reties:
            raise ValueError('Failed to calculate the value of pi for the given bounding range ...')

    pi /=  total_steps

    return pi

In [ ]:
pi_init = 3.141591868192149 # pi_w # 3.1415068949186717
in_points, sq_in_points, total_points = monte_carlo_pi_sampler()

In [ ]:
math.pi - basel_pi_estimator(10000), math.pi - ramanujan_sato_pi_estimator(1)

In [ ]:
pi_out_mc = monte_carlo_pi_estimator(in_points, sq_in_points, total_points)

In [ ]:
pi_mc_rr_ub = riemann_remanan_pi_estimator(
                   in_points,
                   sq_in_points,
                   total_points,
                   num_steps=125000,
                   pi_lower_bound=None,
                   pi_upper_bound=None,
                   num_reties=500 
                 )

In [ ]:
print(math.pi, pi_mc_rr_ub,  pi_out_mc)
print(math.pi - pi_mc_rr_ub, math.pi - pi_out_mc)

In [ ]:
ecc_success = 0
for i in range(100):
    pi_mc_rr_ub = riemann_remanan_pi_estimator(
                      in_points,
                      sq_in_points,
                      total_points,
                      num_steps=12500,
                      pi_lower_bound=None,
                      pi_upper_bound=None,
                      num_reties=500 
                   )
    if np.abs(math.pi - pi_mc_rr_ub) < np.abs(math.pi - pi_out_mc):
        ecc_success += 1

In [ ]:
ecc_success / 100

In [ ]:
pi_out_rr = riemann_remanan_pi_estimator(
                in_points,
                sq_in_points,
                total_points,
                num_reties=1500 
              )

In [ ]:
pi_mc_rr = 0
for i in range(51200):
    pi_mc_rr_list = [pi_out_rr, pi_init]
    pi_mc_rr_min, pi_mc_rr_max = np.min(pi_mc_rr_list), np.max(pi_mc_rr_list)
    pi_mc_rr += random.SystemRandom().uniform(pi_mc_rr_min, pi_mc_rr_max)
pi_mc_rr /= 51200

In [ ]:
print(pi_mc_rr, pi_init, pi_out_rr, pi_out_mc)

In [ ]:
print(math.pi - pi_init, math.pi - pi_mc_rr, math.pi - pi_out_rr, math.pi - pi_out_mc)

## Part 2 -- Using the Buffon's needle problem

In [ ]:
def buffons_pi_estimator(
        num_mc_sims:int=196,
        num_steps:int=196,
        num_repeats:int=196
      )->float:
    '''
       A naive implementation to estimate the value of Pi,                                                   \
         by stochastically solving the Buffon's needle problem.

       Performs placement simulations using a random number of needles with fixed dimensions of length (l),  \
         on a finite two-dimensional plane.

           Arguments of the function:
               num_mc_sims:int -> Number of Monte-Carlo simulations to model the Buffon's needle problem
               num_steps:int   -> Number of parallel equidistant lines along the x axis,                     \
                                    with-in a given two dimenstional spaces;                                 \
                                    of a randomly determined distance (2 * l) value,                         \
                                    used in each step of the Monte-Carlo simulation.
               num_repeats:int -> Number of random needle placements of a randomly determined length (l),    \
                                    for a given 2d sapce.

           Returns a float value corresponding to the approximation of pi.

       This function models the needle placement and rotation through:
         1. Random selection of the x coordinate value for the mid point of the needle.
         2. Random rotation of the needle (θ) by assigning a random value to the x axis projection,        \
              corresponding to the l * cos(θ) component of the needle.
    '''
    num_inter_pts, total_sims = 0, 0

    for i in range(num_mc_sims):
        segm_len  = random.SystemRandom().uniform(1 / num_mc_sims, num_mc_sims)
        rand_orig = random.SystemRandom().uniform(1, num_mc_sims)
    
        x_list = [random.SystemRandom().uniform(-rand_orig, rand_orig)]

        x_list_pos = [x_list[0] + (i * (2 * segm_len)) for i in range(1, num_steps + 1)]
        x_list_neg = [x_list[0] - (i * (2 * segm_len)) for i in range(1, num_steps + 1)]
        
        x_list += x_list_pos
        x_list += x_list_neg

        for j in range(num_repeats):
            x_rand = random.SystemRandom().choice(x_list)

            x_rand_min = x_rand - (random.SystemRandom().uniform(0.73775, 0.83775) * segm_len)
            x_rand_max = x_rand + (random.SystemRandom().uniform(0.73775, 0.83775) * segm_len)
    
            x_mid_segm = random.SystemRandom().uniform(
                             x_rand_min, x_rand_max
                           )

            x_segm_len = random.SystemRandom().uniform(-0.5 * segm_len, 0.5 * segm_len)

            x_pos_segm = x_mid_segm + x_segm_len
            x_neg_segm = x_mid_segm - x_segm_len

            total_sims += 1

            for k, x_k in enumerate(x_list):
                if (x_pos_segm >= x_k and x_neg_segm <= x_k ) or (x_pos_segm <= x_k and x_neg_segm >= x_k ):
                    num_inter_pts += 1
                    break

    pi = total_sims / num_inter_pts

    return pi

${\frac{2.l}{\pi.d}}$

In [ ]:
pi_mc_b_list = [buffons_pi_estimator() for i in range(256)]

## Riemann-Remanan error correction
Error correction using Riemann-Remanan stochastic integration.

In [ ]:
pi_mc_b_min, pi_mc_b_max = np.min(pi_mc_b_list), np.max(pi_mc_b_list)
pi_mc_b = 0
pi_b_lower_bound = 3.1415926538736683
pi_b_upper_bound = 3.1415926538922716
toal_rr_steps = 0
num_retries = 0
while toal_rr_steps == 0:
    num_retries += 1
    for i in range(51200):
        pi_rand_b = random.SystemRandom().uniform(pi_mc_b_min, pi_mc_b_max)
        if pi_b_upper_bound is None or pi_b_lower_bound is None:
            pi_mc_b += pi_rand_b
            toal_rr_steps += 1
        elif pi_rand_b < pi_b_upper_bound and pi_rand_b > pi_b_lower_bound:
            pi_mc_b += pi_rand_b
            toal_rr_steps += 1
    if num_retries > 25000:
        raise ValueError('Failed to compute value of pi within the specified bounding values ...')
pi_mc_b /= toal_rr_steps
print(math.pi, pi_mc_b, np.mean(pi_mc_b_list), math.pi - pi_mc_b, math.pi - np.mean(pi_mc_b_list))